# We will import the necessary libraries

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.models import load_model
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import layers, models, Model, optimizers
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,cohen_kappa_score
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
print('the cell was exectued without error ')

the cell was exectued without error 


In [5]:
#In our Dataset there are 7 categories 
num_classes = 7
#setting the dimensions of images
IMG_SIZE = (224, 224)
#setting batch size to 32 since it will train 32 images at a time
batch_size =     16
#//location of train directory and validation directory 
import pandas as pd
df1=pd.read_csv(r'C:\Users\21260\Projet\Face_Recognition\src\resources\Data\datacsv\Train\Train.csv')
df2=pd.read_csv(r'C:\Users\21260\Projet\Face_Recognition\src\resources\Data\datacsv\Validation\Validation.csv')

labels=['ikram','kabir', 'kannich','khadija','khalid','saad','soufiane' ]
for clas in labels :
    
    df1[clas] = df1['classe'].apply(lambda x: 1 if clas in x else 0)
    df2[clas] = df2['classe'].apply(lambda x: 1 if clas in x else 0)
print('alles Gut')

alles Gut


In [10]:
df1

,path,classe,ikram,kabir,kannich,khadija,khalid,saad,soufiane
0,C:\Users\21260\Projet\Face_Recognition\src\res...,ikram,1,0,0,0,0,0,0
1,C:\Users\21260\Projet\Face_Recognition\src\res...,ikram,1,0,0,0,0,0,0
2,C:\Users\21260\Projet\Face_Recognition\src\res...,ikram,1,0,0,0,0,0,0
3,C:\Users\21260\Projet\Face_Recognition\src\res...,ikram,1,0,0,0,0,0,0
4,C:\Users\21260\Projet\Face_Recognition\src\res...,ikram,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
10495,C:\Users\21260\Projet\Face_Recognition\src\res...,soufiane,0,0,0,0,0,0,1
10496,C:\Users\21260\Projet\Face_Recognition\src\res...,soufiane,0,0,0,0,0,0,1
10497,C:\Users\21260\Projet\Face_Recognition\src\res...,soufiane,0,0,0,0,0,0,1
10498,C:\Users\21260\Projet\Face_Recognition\src\res...,soufiane,0,0,0,0,0,0,1


# Data Augmentation

In [6]:
Train_generator =  ImageDataGenerator (rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#//rescaling of image because for validat
Validation_generator=ImageDataGenerator(rescale=1./255)                             
def flow_dataf(img_generator, dataframe, batch_size):
    g = img_generator.flow_from_dataframe(dataframe,
                                                 x_col='path',
                                                 y_col=labels,
                                                 target_size=IMG_SIZE,
                                                 color_mode='rgb',
                                              class_mode='raw',
                                                 shuffle=False,
                                                 batch_size=batch_size)
    
    return g

train_gen=flow_dataf(img_generator=Train_generator,dataframe=df1,batch_size=16)
validation_gen=flow_dataf(img_generator=Validation_generator,dataframe= df2,batch_size=16)


Found 10500 validated image filenames.
Found 2100 validated image filenames.


In [8]:
img_input=(224, 224, 3)
vgg=VGG16(weights='imagenet', include_top=False,pooling='avg', input_shape=img_input)
#vgg = VGG16(weights="imagenet",include_top=False,pooling='avg',  input_tensor=img_input)
for layer in vgg.layers[:-4]:
    layer.trainable = False
x = vgg.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
prd = Dense(7, activation="softmax")(x)
model = Model(inputs=vgg.input, outputs=prd)
model.summary()

58889256/58889256 [==============================] - 7s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
#ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights(in a checkpoint file)
# at some interval,so the model or weights can be loaded later to continuethe training from the state saved.
checkpoint = ModelCheckpoint('C:\Users\21260\Projet\Face_Recognition\src\CNN\final_poids_VGG_16.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
#//earlystop will be applied for where there is no improvement in validation loss upto 3 epochs
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)
#//reduce_lr will also monitor the validation loss 
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=3,
                              verbose=1,
                              min_delta=0.000001)
callb = [earlystop, checkpoint, reduce_lr]
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=.0001),
              metrics=['accuracy'])
call=[ checkpoint,reduce_lr]

# We will training our model

In [ ]:
history = model.fit_generator(generator=train_gen,steps_per_epoch=train_gen.n/batch_size, epochs=3 ,
    validation_data=validation_gen,
    validation_steps=validation_gen.n/batch_size,callbacks=call)

# We will print the history of

In [ ]:
print(history.history.keys()) 
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left') 
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

In [ ]:
model.save('C:\Users\21260\Projet\Face_Recognition\src\CNN\final_model_VGG_16.h5')

In [ ]:
model = load_model("C:\Users\21260\Projet\Face_Recognition\src\CNN\final_model_VGG_16.h5")
model.load_weights("C:\Users\21260\Projet\Face_Recognition\src\CNN\final_poids_VGG_16.h5")

T=2100
batch_size=128
import numpy as np

predictions = model.predict_generator(validation_generator,steps= validation_generator.n/validation_generator.batch_size,verbose = True)
y_pred = np.argmax(predictions, axis=1)

true_classes = validation_generator.classes

class_labels = list(validation_generator.class_indices.keys())   

print(class_labels)
print(confusion_matrix(validation_generator.classes, y_pred))
report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)
print("f1  :",f1_score(true_classes, y_pred,average='macro'))
print("accuracy VGG16 :",accuracy_score(np.argmax(validation_generator.classes, axis=1), np.argmax(y_pred, axis=1)))